In [1]:
import pandas as pd
import os

from gh_logging import LogManager


def is_igh(obj, igh):
    """Check to see if an object implements a given interface, for example IGH_Component or IGH_Params
        ::return True if the object implements a given interface
    """
    try: 
        if igh(obj):
            return True
    except:
        return False
    
def split_gh_graph_numbering(string):
    return [int(x) for x in string.split("-")]



In [2]:
import pandas as pd
pd.read_csv("grasshopper_components.csv")

,category,name,guid,nickname,description,type,library
0,Curve,Arc,bb59bffc-f54c-4682-9778-f6c3fe74fce3,Arc,"Create an arc defined by base plane, radius an...",CurveComponents.Component_Arc,Curve Components
1,Curve,Arc 3Pt,9fa1b081-b1c7-4a12-a163-0aa8da9ff6c4,Arc,Create an arc through three points.,CurveComponents.Component_Arc3Pt,Curve Components
2,Curve,Arc SED,9d2583dd-6cf5-497c-8c40-c9a290598396,Arc,"Create an arc defined by start point, end poin...",CurveComponents.Component_ArcSED,Curve Components
3,Curve,Bezier Span,30ce59ce-22a1-49ee-9e21-e6d16b3684a8,BzSpan,Construct a bezier span from endpoints and tan...,CurveComponents.Component_BezierSpan,Curve Components
4,Curve,BiArc,75f4b0fd-9721-47b1-99e7-9c098b342e67,BiArc,Create a bi-arc based on endpoints and tangents.,CurveComponents.Component_BiArc,Curve Components
...,...,...,...,...,...,...,...
1729,eleFront,Update All Reference Components,63eaf9fd-7214-475a-8187-20a458a69138,Update Refs - All,Update all eleFront Reference components with ...,EleFront.Components.Reference.Param_UpdateAllR...,eleFront
1730,eleFront,Update Group,7f88e70f-e456-40ee-a795-78fbaceb82ce,Update Group,Activate all EleFront components that share th...,EleFront.Components.Reference.Param_UpdateGrou...,eleFront
1731,eleFront,eleFront Block,877a63a0-03b5-4c4c-90f9-65f1b7f86f65,Block,Contains eleFront Block,EleFront.Components.Params.Param_Block,eleFront
1732,eleFront,eleFront Data Description,5bd835d3-a4c1-4922-839b-3a9ebeb78b67,Description,Use this component to guide your data streams ...,EleFront.Components.Params.Param_DataDesc,eleFront


In [1]:
from pathlib import Path

logger1 - INFO - This is an info message for logger1.


complog - DEBUG - This is a debug message from complog
complog - DEBUG - This is a debug message from complog
complog - DEBUG - This is a debug message from complog


complog - DEBUG - This is a debug message from complog
complog - DEBUG - This is a debug message from complog
complog - DEBUG - This is a debug message from complog


In [23]:
import datetime
import json
import os
import pickle
import shutil
from pathlib import Path
from gh_logging import LogManager
class EnvironmentManager:
    _environments = {}  # Stores Environment instances keyed by environment_name

    # Define directory structure as a class-level attribute.
    DIR_STRUCTURE = {
        "vanilla": "00-VanillaComponents",
        "all": "01-AllComponents",
        "logs": "02-Logs",
        "files": "03-GH_Files",
        "csv": "04-CSV",
        "graphml": "05-GraphML",
        "ghlib": "99-GH-Lib"
    }

    GHPATH = r"C:\Users\jossi\AppData\Roaming\Grasshopper\Libraries"
    VANILLA_PATH = r"C:\Users\jossi\Dropbox\Office_Work\Jos\GH_Graph_Learning\Grasshopper Components\240307-CoreComponents\vanilla_components.csv"
   
    class Environment:
        def __init__(self, environment_name):
            prefix = datetime.datetime.now().strftime("%y%m%d")
            self.environment_name = environment_name
            self.base_path = Path("ExtractionEnvironments") / f"{prefix}-{environment_name}"
            self.base_path.mkdir(parents=True, exist_ok=True)
            self.logmanager = None
            # Initialize directories based on the class-level DIR_STRUCTURE
            self.dirs = {name: self.base_path / dirname for name, dirname in EnvironmentManager.DIR_STRUCTURE.items()}
            for path in self.dirs.values():
                path.mkdir(parents=True, exist_ok=True)
            #  Create loggers.
            LogManager.new_logger("clog", Path(self.base_path)/self.dirs['logs'], "clog.log")
            LogManager.new_logger("flog", Path(self.base_path)/self.dirs['logs'], 'flog.log')
            self.gh_path = Path(EnvironmentManager.GHPATH)
            self.initialise()
            self.serialize()
            

        def serialize(self):
            file_path = self.base_path / "environment.pkl"
            with open(file_path, "wb") as file:
                pickle.dump(self, file)

        def clone_env(self, clone=False):
            GH_path = Path(self.gh_path)  # Ensure GH_path is a Path object
            lib_path = self.dirs.get('ghlib')

            if clone:
                # os.walk() iterates over the directories and files in GH_path
                for root, dirs, files in os.walk(GH_path):
                    root_path = Path(root)
                    relative_path = root_path.relative_to(GH_path)
                    # For each file in the current directory
                    for file in files:
                        # Source file path
                        source_path = root_path / file
                        # Destination path includes the relative_path to preserve the directory structure
                        destination_path = lib_path / relative_path / file
                        # Ensure the destination directory exists
                        destination_path.parent.mkdir(parents=True, exist_ok=True)
                        # Copy the file to the destination
                        shutil.copy(source_path, destination_path)

        def reinstate_env(self, override=False):
            if input("Are you sure you want to reinstate the environment? \n"
                     "This will replace existing files in GH Libraries Folder"
                     "with those belonging to this environment? [y/n]").lower() == "y" \
                    or override:
                print("Reinstating environment...")
                GH_path = self.gh_path
                lib_path = self.dirs["99-GH-Lib"]

                # Delete all files in GH_path
                for root, dirs, files in os.walk(GH_path):
                    for file in files:
                        file_path = Path(root) / file
                        if file_path.is_file():
                            file_path.unlink()

                # Copy all files from 99-GH-Lib back to GH_path
                for file in lib_path.iterdir():
                    if file.is_file():
                        shutil.copy(file, GH_path)
            else:
                print("Reinstate cancelled")
    
        def clone_env(self, clone=False):
            GH_path = Path(self.gh_path)  # Ensure GH_path is a Path object
            lib_path = self.dirs["ghlib"]
    
            if clone:
                # os.walk() iterates over the directories and files in GH_path
                for root, dirs, files in os.walk(GH_path):
                    # Convert root to a Path object for easier manipulation
                    root_path = Path(root)
    
                    # Calculate the relative path from GH_path to the current directory
                    relative_path = root_path.relative_to(GH_path)
    
                    # For each file in the current directory
                    for file in files:
                        # Source file path
                        source_path = root_path / file
    
                        # Destination path includes the relative_path to preserve the directory structure
                        destination_path = lib_path / relative_path / file
    
                        # Ensure the destination directory exists
                        destination_path.parent.mkdir(parents=True, exist_ok=True)
    
                        # Copy the file to the destination
                        shutil.copy(source_path, destination_path)
    
        def reinstate_env(self, override = False):
            if input("Are you sure you want to reinstate the environment? \n"
                     "This will replace existing files in GH Libraries Folder"
                     "with those belonging to this environment? [y/n]").lower() == "y"\
                    or override:
                print("Reinstating environment...")
                GH_path = self.gh_path
                lib_path = self.dirs["ghlib"]
    
                # Delete all files in GH_path
                for root, dirs, files in os.walk(GH_path):
                    for file in files:
                        file_path = Path(root) / file
                        if file_path.is_file():
                            file_path.unlink()
    
                # Copy all files from 99-GH-Lib back to GH_path
                for file in lib_path.iterdir():
                    if file.is_file():
                        shutil.copy(file, GH_path)
            else:
                print("Reinstate cancelled")
    
        def view_gh_environment(self):
            for root, dirs, files in os.walk(self.dirs["ghlib"]):
                for file in files:
                    print(file)
    
        def copy_file(self, src, dst):
            """
            Copies a file from the source path to the destination path.
    
            :param src: The source file path as a string or Path object.
            :param dst: The destination file path as a string or Path object.
            """
            # Ensure that the source exists and is a file
            if not src.exists() or not src.is_file():
                print(f"Source file does not exist or is not a file: {src}")
                return
    
            # Ensure the destination directory exists, if not, create it
            dst.parent.mkdir(parents=True, exist_ok=True)
    
            # Copy the file
            try:
                shutil.copyfile(src, dst)
                print(f"File copied successfully from {src} to {dst}.")
            except Exception as e:
                print(f"Error copying file from {src} to {dst}: {e}")
        
        def initialise(self, vanilla_components: Path = None, clone_env=True):
            print("Setting environment variables")
            print("Copying vanilla components")
            if vanilla_components is None:
                vanilla_components = Path(EnvironmentManager.VANILLA_PATH)
            if vanilla_components:
                # Ensure the destination directory exists
                vanilla_components_destination = self.dirs["vanilla"]
                vanilla_components_destination.mkdir(parents=True, exist_ok=True)
                # Construct the full destination path including the filename
                destination_path = vanilla_components_destination / vanilla_components.name
                # Use the adjusted destination path that includes the filename
                self.copy_file(vanilla_components, destination_path)
            print("Copying components")
            self.clone_env(clone_env)
        
    
        def get_test_files(self):
            for root, dirs, files in os.walk(self.dirs["files"]):
                for file in files:
                    if file.endswith(".gh"):
                        yield os.path.join(root, file)


    @classmethod
    def create_or_load_environment(cls, environment_name):
        if environment_name not in cls._environments:
            cls._environments[environment_name] = cls.Environment(environment_name)
        return cls._environments[environment_name]

    @classmethod
    def get_directory_names(cls):
        # Returns the directory names defined at the class level
        return list(cls.DIR_STRUCTURE.values())





NameError: name '__file__' is not defined